In [ ]:
# +++++++++++++++++++++++++++
#  Studing Collisonal Methods
# +++++++++++++++++++++++++++

import numpy as np
from scipy.linalg import expm
import matplotlib.pyplot as plt
from qutip import *
import pickle
import os

# ===========================================================================================

sz = np.array([[1,0], [0,-1]]); sx = np.array([[0,1],[1,0]]); sy = np.array([[0,-1j],[1j,0]]) 

# ===========================================================================================

# ===================
#  Lindblad functions
# ===================

def Liouvillian(H, gamma_k, L_k):
    """
    Build the Liouvillian superoperator.
    
    Parameters: - H : ndarray, Hamiltonian matrix
                - gamma_k : list, Decay rates
                - L_k : list, Jump Operators
    
    Returns: - super_L : ndarray, Liouvillian superoperator
    """    
    I = np.eye(H.shape[0])
    super_L = -1.j * (np.kron(I, H) - np.kron(H.T, I))
    
    for k in range(len(gamma_k)):
        super_L += gamma_k[k] * (np.kron(np.conj(L_k[k]), L_k[k]) - 0.5 * np.kron(I, np.conj(L_k[k]).T @ L_k[k]) - 0.5 * np.kron((np.conj(L_k[k]).T @ L_k[k]).T, I))
    
    return super_L

# ====================================================================================

def Lindblad_evo(rho, H, gamma_k, L_k, times, method="expm", vectorized=True):
    """
    Evolution of the density matrix with the Linblad Eq.
    
    Method: - "U" -> propagator = expm(super_L * dt) (needs L_k as NumPy array)
            - "diagonal" -> diagonalizzation of the super-op. (needs L_k as NumPy array )
        
    Vectorized: True/False to choose the output format

    Parameters: - H : Qobj or ndarray, System Hamiltonian
                - rho : Qobj or ndarray, Initial Density Matrix
                - gamma_k : list, List of Decay Rates
                - L_k : list, List of Jump Operators
                - times : array, Time array
        
    Returns : - if vectorized=True → array (N^2, Nt)
              - if vectorized=False → array (Nt, N_site, N_site)
    """
    rho_shape = H.shape[0]
    dt = times[1] - times[0]
   
    # Converts Qobj in NumPy array 
    L_k = [L.full() if hasattr(L, "full") else np.array(L) for L in L_k]
    H = H.full() if hasattr(H, "full") else np.array(H)
    rho = rho.full() if hasattr(rho, "full") else np.array(rho)
        
    # Build up of the Lindbladian
    super_L = Liouvillian(H, gamma_k, L_k)
       
    # Vectorized intial state
    rho_vec = rho.reshape(rho_shape * rho_shape)
        
    # Result array inizialized
    rho_vec_list = np.zeros((rho_shape * rho_shape, len(times)), dtype=complex)
    rho_vec_list[:, 0] = rho_vec

    # -------------
    # Expm method
    # -------------

    # Time Propagator
    if method == "expm":
        super_U = expm(super_L * dt)
            
        # Evolution
        for i in range(1, len(times)):
            rho_vec_list[:, i] = super_U @ rho_vec_list[:, i - 1]
            
        # Output
        if vectorized:
            return rho_vec_list  # (rho_shape^2, Nt)
        else:
            return rho_vec_list.T.reshape(len(times), rho_shape, rho_shape)  # (Nt, rho_shape, rho_shape)

    # ------------------
    # Diagonal method
    # ------------------    
             
    elif method == "diagonal":  
         
        #Diagonalization of the Super-Op.
        W, V = np.linalg.eig(super_L)
        V_inv = np.linalg.inv(V)
                    
        # Build up of the Diagonal Lindbladian
        U_diag = np.exp(W * dt)

        # Initial coefficients in the eigenvectors base
        coeff = V_inv @ rho_vec        
        coeff_list = np.zeros((len(W), len(times)), dtype=complex)
        coeff_list[:, 0] = coeff
           
        # Evolution of the coefficients
        for i in range(1, len(times)):
            coeff_list[:, i] = U_diag * coeff_list[:, i - 1]  

        # Reconversion in the original base
        rho_vec_list = V @ coeff_list
                
        # Output with Eigenenergies & Eigenvectors (as NumPy array)
        if vectorized:
            return rho_vec_list, V, W  
        else:
            return rho_vec_list.T.reshape(len(times), rho_shape, rho_shape), V, W 

    else:
        raise ValueError("Set 'expm', 'diagonal'.")

# =================================================================================================================

def compute_Lindblad(H_system, rho_sys_initial, gamma_k, L_k, times):
    """
    Compute Lindblad evolution using diagonal method.
    
    Parameters: - H_system : Qobj or ndarray, System Hamiltonian
                - rho_sys_initial : Qobj or ndarray, Initial Density Matrix
                - gamma_k : list, List of Decay Rates
                - L_k : list, List of Jump Operators
                - times : array, Time array
    
    Returns: - rho_list_lindblad : ndarray, Density matrix evolution (len(times) x N_site x N_site)
             - V : ndarray, Eigenvectors of the Liouvillian
             - W : ndarray, Eigenvalues of the Liouvillian
    """
    
    rho_list_lindblad, V, W = Lindblad_evo(rho_sys_initial, H_system, gamma_k, L_k, times, method="diagonal", vectorized=False)

    return rho_list_lindblad, V, W

# =================================================================================================================

# =============================
#  Collisional Method functions
# =============================

def system_Hamiltonian(N_site, E, V_pot, mode="complete"):
    """
    Build up of the System's Hamiltonian for the complete basis (ground & excited states) or only excited states.
    
    Method: - "complete"-> complete basis (ground & excited states)
            - "exc"-> excited basis (only excited states)
    
    Parameters: - E: Float, System's Site Energies (randomly generated)
                - V_pot: Float, Hopping Potential
                - N_site : Int, Number of Sites
        
    Returns : System's Hamiltonian as Numpy array
    """
    # -------------------------
    # Only Excited States Basis
    # -------------------------
    if mode == "exc":   
        H_sys = np.zeros((N_site, N_site), dtype=complex)
        for i in range(N_site):
            H_sys[i, i] = E[i]
        for i in range(N_site):
            for j in range(N_site):
                if i != j:
                    H_sys[i, j] = V_pot
        return H_sys
        
    # --------------
    # Complete Basis 
    # --------------    
    elif mode == "complete":   
        H_sys = np.zeros((2**N_site, 2**N_site), dtype='complex')
        
        for i in range(N_site):
            H_i = (E[i]/2) * (tensor(identity(2**i), identity(2)-sigmaz(), identity(2**(N_site-i-1)))) # g states have 0 energy and the e sites have all the energy
            H_sys += H_i.full()

            for j in range(i+1, N_site):
               H_ij = V_pot/2 * (tensor(identity(2**i), sigmax(), identity(2**(j-i-1)), sigmax(), identity(2**(N_site-j-1))) + tensor(identity(2**i), sigmay(), identity(2**(j-i-1)), sigmay(), identity(2**(N_site-j-1))))
               H_sys += H_ij.full()
        
        return H_sys

    else:
        raise ValueError("mode has to be 'complete' or 'exc'")

# ===============================================================================================================================================

def interaction_Hamiltonian_N_ancillas(N_site, c_CM):   #Hamiltonian of interaction with N different ancillas, one for every site of the system
    """
    Build up of the Hamiltonian of Interaction for the Collision System - Ancilla
       
    Parameters: - N_site : int, Number of Sites
                - c_CM : list, Interaction Forces for the System - Ancilla intercation/collsion
        
    Returns : Hamiltonian of Interaction as Qutip object
    """
    dim_tot = 2**(2 * N_site)
    H_int = np.zeros((dim_tot, dim_tot), dtype=complex)   #inizialization

    for j in range(N_site):
  
        op_list = [identity(2) for _ in range(2 * N_site)]  #list of identity to be fill with the operator sigmaz & sigmax; 2N identity, N for the system and N fo the ancillas
        
        op_list[j] = sigmaz()      # Acts on the j site
        op_list[N_site + j] = sigmax()  # Acts on the j ancilla, with index N + j
        
        H_term = (c_CM[j] * tensor(op_list)).full()  # tensor product between the element of the list
        
        H_int += H_term

    return H_int

# ================================================================================================================================================================================
    
def hamiltonian_N_ancillas(N_site, E, V_pot, c_CM):
    """
    Generation of 3 Hamiltonians in Qutip format for the collision model with N ancillas:
                - H_system : system Hamiltonian
                - H_collision : interaction Hamiltonian with N ancillas
                - H_tot : complete Hamiltonian (system + collision)

    Parameters: - E: Float, System's Site Energies (randomly generated)
                - V_pot: Float, Hopping Potential
                - N_site : int, Number of Sites
                - c_CM : list, Interaction Forces for the System - Ancilla intercation/collsion
    
    Returns : H_system, H_collision, H_tot as Qutip object
    """
    H_collision = interaction_Hamiltonian_N_ancillas(N_site, c_CM)
    
    H_system = system_Hamiltonian(N_site, E, V_pot, mode="complete")
        
    dim_anc = 2**N_site
    Id_ancillas = np.eye(dim_anc, dtype=complex)
    H_system_expanded = np.kron(H_system, Id_ancillas)  #expand H_sys in the total space
    
    H_tot = H_system_expanded + H_collision
        
    return H_system, H_collision, H_tot

#====================================================================

def evolution_operator(H, dt, method='expm', hermitian=True):
    """
    Build up of the evolution operator U = exp(-i H dt) using Expm or analytic diagonalization.
   
    Parameters: - H : Np array, System Hamiltonian
                - dt : float, Timestep
    
    Method : - "expm"-> build up of the Matrix Exponential with expm
             - "diagonalization"->  build up of the propagater U as V @(exp(-i W dt))@ V_dag with W eigenvalues and V eigenvector of the Hamiltonian 

    Returns : Evolution Operator U, 
    """
    H = H.full() if hasattr(H, "full") else np.array(H)
    
    # -----------
    # Expm method
    # -----------
    
    if method == 'expm':
        U = expm(-1j * H * dt)
        return U
        
    # ---------------
    # Diagonalization
    # ---------------
    
    elif method == 'diagonalization':
        if hermitian:
            w, V = np.linalg.eigh(H)
            V_inv = V.conj().T
        else:
            w, V = np.linalg.eig(H) 
            V_inv = np.linalg.inv(V)
                
        U_diag = np.diag(np.exp(-1j * w * dt))
        U = V @ U_diag @ V_inv
        return U, U_diag, w, V

    else:
        raise ValueError("method must be 'expm' or 'diagonalization'")

#====================================================================================

# ========================================================
#  Evolution with U_complete and then trace on the ancilla
# ========================================================

def compute_trace_ancilla(rho_sys_initial, rho_anc_all, U_diag, V, times, P_10, P_01, N_site):
    """
    Compute time evolution with Trace over Ancilla and Reset, using Density Matrix formalism.; equal to Infinite Trajectories

    Parameters: - rho_sys_initial : Np array, Initial Density Matrix
                - rho_anc_all : Np array, Initial Density Matrix of the Ancilla expanded in the total Hilber Space (S + A)
                - U_diag: Np array, Time Evolution Operator for the complete system (S + A) in diagonal form
                - U_diag_dag: Np array, Adjoint operator of U_diag
                - V_diag: Np array, Hopping Potential
                - V_diag_dag: Np array, Adjoint operator of V_diag
                - times : array, Time array
                - N_site : int, Number of Sites                
                - P_10, P_01 : Np array, Projection Operators on |10> & |01>
                
    Returns: - pops_complete : ndarray (N_site x len(times)),  Population evolution for each site
    """
    # check and conversion in Numpy object
    rho_sys = rho_sys_initial.full() if hasattr(rho_sys_initial, 'full') else rho_sys_initial.copy()
    rho_anc = rho_anc_all.full() if hasattr(rho_anc_all, 'full') else rho_anc_all.copy()

    # Dimension of the density matrix for the partial trace
    dim_sys = rho_sys.shape[0]
    dim_anc = rho_anc.shape[0]

    # Evolution Operator 
    U_step = V @ U_diag @ V.conj().T; U_step_dag = U_step.conj().T
    
    # Array to store the results of the evolution for only the excited site
    pops_complete = np.zeros((N_site, len(times)), dtype=float) 
   
    #initial state
    pops_complete[0, 0] = np.real(np.trace(P_10 @ rho_sys))
    pops_complete[1, 0] = np.real(np.trace(P_01 @ rho_sys)) 
    
    # Time Evolution
    for t in range (1, len(times)):
    
        # 1 : expansion in the system-ancilla space by tensor product
        rho_tot = np.kron(rho_sys, rho_anc)   # I always use ancillas in their initial state! already resetted 
        
        # 2 : Unitary evolution of the total rho
        rho_tot = U_step @ rho_tot @ U_step_dag
        
        # 3 : Partial Trace on the ancilla's degree of freedom
        rho_tot_reshaped = rho_tot.reshape(dim_sys, dim_anc, dim_sys, dim_anc) # 4 blocks 4x4 (tensor)
        rho_sys = np.trace(rho_tot_reshaped, axis1=1, axis2=3) # contraction over elements with same ancilla's index
    
        # 4 : Store the result of the population
        pops_complete[0, t] = np.real(np.trace(P_10 @ rho_sys))
        pops_complete[1, t] = np.real(np.trace(P_01 @ rho_sys))

    return pops_complete

#================================================================================================================

# =================
# Single trajectory
# =================

def compute_trajectory_wf(dt, c_CM, N_traj, N_site, times, P_10, P_01, psi_sys_initial, U_site):
    """
    Compute quantum trajectory evolution of the wave function with a collisional algorithm.
    
    Parameters: - dt : float, Time Step
                - c_CM_list : array, Collisional model Coefficients
                - N_traj : int, Number of Trajectories
                - N_site : int, Number of Sites
                - steps : int, Number of Time Steps
                - times : array, Time array
                - P_10, P_01 : Np array, Projection Operators on |10> & |01>
                - psi_sys_initial : Qobj, Initial Wave Function
                - U_site : Qobj, Time Evolution Operator
                       
    Returns: -  pop_traj : ndarray, Population for each trajectory (N_site x steps x N_traj)
              -  average_pop_traj : ndarray, Average population over trajectories (N_site x steps)
    """   
    # check and conversion in Numpy object
    U_site = U_site.full() if hasattr(U_site, 'full') else U_site
    psi_sys_initial = psi_sys_initial.full() if hasattr(psi_sys_initial, 'full') else psi_sys_initial.copy()
    
    # Evolution of the Wave Function
    jump_probabilities = [np.sin(c * dt)**2 for c in c_CM]   #probability of appling sigmaz to psi
    
    count = np.zeros(N_traj)
    pop_traj = np.zeros((N_site, len(times), N_traj))  #array to store the population of every site at every step of evolution for every trajectories
   
    # Costruction of the sigmaz operator for every site
    Sz_ops = []
    for idx in range(N_site):
        ops = [qeye(2) for _ in range(N_site)]
        ops[idx] = sigmaz()  # sigmaz only on teh index site
        Sz_op = tensor(ops).full()
        Sz_ops.append(Sz_op)

    Sz_ops = np.array(Sz_ops)
        
    # Inizialization
    pop_site_1_initial = np.real(np.vdot(psi_sys_initial, P_10 @ psi_sys_initial))
    pop_site_2_initial = np.real(np.vdot(psi_sys_initial, P_01 @ psi_sys_initial))
    
    pop_traj[0,0,:] = pop_site_1_initial
    pop_traj[1,0,:] = pop_site_2_initial
   
    # Evolution for different Trajectories 
    for traj in range (N_traj):
        # 1 reset initial wf |01>
        psi = psi_sys_initial.copy()   
        
        # 2 Evolution in Time (starting after 1st timestep)
        for step in range(1, len(times)):
            
            psi = U_site @ psi   # Evolution of wf with H_site
                       
            # 3 Defining the condition of the Monte Carlo - Jump for every site
            for site_index in range(N_site):
                rn_site = np.random.rand() # Random number between 0 & 1
                
                if rn_site < jump_probabilities[site_index]:
                    psi = Sz_ops[site_index] @ psi   # apply sigmaz 
                    #psi = psi / psi.norm()   #{No renormalization, sigmaz unitary}
                    
                    count[traj] = count[traj] + 1  # count number of collision
            
            # 4 Expectation Value
            pop_site_1 = np.real(np.vdot(psi, P_10 @ psi))
            pop_site_2 = np.real(np.vdot(psi, P_01 @ psi))
            
            pop_traj[0, step, traj] = pop_site_1
            pop_traj[1, step, traj] = pop_site_2
    
    # Average value of different trajectories
    average_pop_traj = np.mean(pop_traj, axis=2)

    return pop_traj, average_pop_traj, count

# ===============================================================================================================

# ===================================
# Count number of effective collision
# ===================================

def compute_count(count, steps):
    """
    Normalization of the count of the Effective collision over the number of time steps and calculation of the average probability of collsion

    Parameters: - count : array, Number of Inneficient Collision (that applies identity to the system wave function)
                - steps : int, number of Time Steps
                
    Returns : - count : same matrix Normalized and with number of Effective Collision (that applies sigmaz to the system wave function)
              - media : average probabilty to have an Effective Collision calculated over N trajectories
    """
    
    for x in range (0, len(count)):
        count[x] /= steps
    
    sum = 0.0
    for x in range (0, len(count)):
        sum += count[x]
    media = sum / (len(count))

    return count, media 

# =================================================================================================================

# ================
# Isolated system
# ================

def compute_trajectory_wf_isolated(N_site, times, P_10, P_01, psi_sys_initial, U_site):
    """
    Compute quantum trajectory evolution of the wave function with only H_system = Energy of the Site and Hopping Potential V.
    Calculated for only 1 trajecotry (always the same evolution)
    
    Parameters: - N_site : int, Number of Sites
                - steps : int, Number of Time Steps
                - times : array, Time array
                - P_10, P_01 : Np array, Projection Operators on |10> & |01>
                - psi_sys_initial : Qobj, Initial Wave Function
                - U_site : Qobj, Time Evolution Operator
                       
    Returns: -  pop_traj_isolated : ndarray, Population for each trajectory (N_site x steps x N_traj)
             -  average_pop_traj_isolated : ndarray, Average population over trajectories (N_site x steps)
    """

    # check and conversion in Numpy object
    U_site = U_site.full() if hasattr(U_site, 'full') else U_site
    psi_sys_initial = psi_sys_initial.full() if hasattr(psi_sys_initial, 'full') else psi_sys_initial.copy()
    
    # Isolated system
    pop_traj_isolated = np.zeros((N_site, len(times)))
    
    # Inizialization
    pop_site_1_initial = np.real(np.vdot(psi_sys_initial, P_10 @ psi_sys_initial))
    pop_site_2_initial = np.real(np.vdot(psi_sys_initial, P_01 @ psi_sys_initial))
    
    pop_traj_isolated[0,0] = pop_site_1_initial
    pop_traj_isolated[1,0] = pop_site_2_initial
        
    psi = psi_sys_initial.copy()   #reset initial wf |01>
    
    # Evolution in Time (starting after 1st timestep)
    for step in range(1, len(times)):
            
        psi = U_site @ psi   # Evolution of wf with H_site
                   
        pop_site_1 = np.real(np.vdot(psi, P_10 @ psi))
        pop_site_2 = np.real(np.vdot(psi, P_01 @ psi))
    
        pop_traj_isolated[0, step] = pop_site_1
        pop_traj_isolated [1, step] = pop_site_2
    
    # Average value = single traj
    average_pop_isolated = pop_traj_isolated 

    return pop_traj_isolated, average_pop_isolated

# ===================================================================================================================================================

# ===================================
# Main Loop for varying dt and N_traj
# ===================================

# System's Parameters
N_site = 2            # Number of sites
V_pot = 1.0           # Hopping Potential
E = 1.5 + np.random.randn(N_site)*0.1     #random inizialization of the system energies

# Time Evolution Parameters
dt_list = [0.01, 0.02, 0.05, 0.1 ]   # Time step
tf = 50.0    # Final Time
steps_list = [ int(tf / dt_list[i]) for i in range (len(dt_list)) ]
times_list = [ np.linspace(0, tf, int(steps_list[i])) for i in range(len(dt_list))]

N_traj_list = [100, 1000]

# Dephasing Parameter (come in MATLAB)
g_deph = 0.1  # Gamma rate

# Scaling for the collsional algorithm c = sqrt(gamma / 4dt)
c_CM_list = np.array([[np.sqrt(g_deph / (4 * dt_list[j])) for j in range(len(dt_list))] for _ in range(N_site)])  # same Coupling for the 2 sites

#Lindblad Rates
gamma_k = [g_deph, g_deph]

# Initial wave function and density matrix
# System
psi_sys_initial = tensor(basis(2, 0), basis(2, 1)) # I set the population only in site 2
rho_sys_initial = (ket2dm(psi_sys_initial)).full()

# Ancilla
rho_anc_single = ket2dm(basis(2, 0)) # Pure state |0><0| for a singe ancilla
rho_anc_all = (tensor([rho_anc_single for _ in range(N_site)])).full() #for N ancilla

# Projector for the extraction from the rho
P0 = (np.eye(2) + sz) / 2 # projector on |0>
P1 = (np.eye(2) - sz) / 2 # projector su |1>

P_00 = np.kron(P0, P0) # |00><00|
P_01 = np.kron(P0, P1) # |01><01|
P_10 = np.kron(P1, P0) # |10><10|
P_11 = np.kron(P1, P1) # |11><11|

#Lindblad Jump Operator
L_1 = P_10  # projector on |10><10|
L_2 = P_01  # projector on |01><01|
L_k = [L_1, L_2]

# ===========
# Calculation
# ===========

# Dictionary for results
results = {}

print("Starting computation for different dt and N_traj")

# Loop over dt
for dt_idx, dt in enumerate(dt_list):

    # Initialize dictionary for this dt
    results[dt] = {}
    
    print("=" * 40)
    print(f"Processing dt = {dt:.4f} ")
        
    # Extract parameters for this dt
    times = times_list[dt_idx]
    steps = steps_list[dt_idx]
    c_CM = c_CM_list[:, dt_idx]  # Column corresponding to this dt
    
    # =====================================
    # Recalculate Hamiltonian and Operators
    # =====================================
    print("Recalculating Hamiltonians")       #because c_CM depends on dt
    
    # Hamiltonian & U for Density Matrix (with ancillas)
    H_site, H_coll, H_tot = hamiltonian_N_ancillas(N_site, E, V_pot, c_CM)
    
    U_tot, U_diag, w, V = evolution_operator(H_tot, dt, method='diagonalization', hermitian=True)
    U_diag_dag = U_diag.conj().T; V_dag = V.conj().T 
    
    # Hamiltonian & U for Wave Function (system only)
    H_system = system_Hamiltonian(N_site, E, V_pot, mode="complete")
    
    U_site, U_diag_site, w_site, V_site = evolution_operator(H_system, dt, method='diagonalization', hermitian=True)
    
    # =========
    # Lindblad
    # =========
    print("Computing Lindblad")
        
    rho_list_lindblad, V_lindblad, W_lindblad = compute_Lindblad(H_system, rho_sys_initial, gamma_k, L_k, times)
    
    # ==============
    # Trace Ancilla
    # ==============
    print("Computing Trace Ancilla")
    
    pops_trace = compute_trace_ancilla(rho_sys_initial, rho_anc_all, U_diag, V, times, P_10, P_01, N_site)

    # ========================================
    # Trajectory Isolated (without collisions)
    # ========================================
    print("Computing Trajectory Isolated")
        
    pop_traj_isolated, average_pop_isolated = compute_trajectory_wf_isolated(N_site, times, P_10, P_01, psi_sys_initial, U_site)
    
    # ================
    # Loop over N_traj
    # ================
    for N_traj in N_traj_list:
        
        print("-" * 40)
        print(f"Processing N_traj = {N_traj}")
                
        # ===================================
        # Trajectory for WF (with collisions)
        # ===================================
        print("Computing Trajectory WF")
             
        pop_traj, average_pop_traj, count = compute_trajectory_wf(dt, c_CM, N_traj, N_site, times, P_10, P_01, psi_sys_initial, U_site)
           
        # Calculate collision statistics
        count_normalized, collision_prob = compute_count(count.copy(), steps)  # copy count to be reused
                
        # ==================================
        # Save the Results in the Dictionary
        # ==================================

        n_samples_to_save = 3 # number of single exemplary trajectories to store
        indices = np.random.choice(pop_traj.shape[2], n_samples_to_save, replace=False) # random extraction of indices
        traj_samples = pop_traj[:, :, indices]
        
        results[dt][N_traj] = {
            'parameters': {
                'dt': dt,
                'N_traj': N_traj,
                'times': times,
                'steps': steps,
                'c_CM': c_CM.copy()  
            },
            
            # Trace Ancilla 
            'pops_trace': pops_trace,

             # Trajectory Isolated
            'trajectory_isolated': {
                'pop_traj': pop_traj_isolated,
                'average_pop': average_pop_isolated
            },
            
            # Lindblad 
            'lindblad': {
                'rho_list': rho_list_lindblad,
                'V': V_lindblad,
                'W': W_lindblad
            },

            # Trajectory WF
            'trajectory_wf': {
                'pop_traj_samples': traj_samples,  # (N_site x len(times) x n_samples_to_save)
                'average_pop': average_pop_traj,  # (N_site x len(times))
                'count': count,
                'count_normalized': count_normalized,
                'collision_prob': collision_prob
            }        
        }
        
        print(f"Results saved in results[{dt}][{N_traj}]")
                
print("\n" + "=" * 22)
print("Calculation Completed")
print("=" * 22)


In [ ]:
times_1000 = results[0.01][1000]['parameters']['times']
pop_trace_1000 = results[0.01][1000]['pops_trace']
pop_avg_traj_1000 = results[0.01][1000]['trajectory_wf']['average_pop']
rho_lindblad_1000 = results[0.01][1000]['lindblad']['rho_list']

times_100 = results[0.01][100]['parameters']['times']
pop_trace_100 = results[0.01][100]['pops_trace']
pop_avg_traj_100 = results[0.01][100]['trajectory_wf']['average_pop']
rho_lindblad_100 = results[0.01][100]['lindblad']['rho_list']


fig01, ax = plt.subplots(figsize=(10, 5))

ax.plot(times_100, pop_trace_100[0,:], label=r'Trace Ancilla Site 1', linewidth=2, linestyle='--')
ax.plot(times_100, np.real(rho_lindblad_100[:, 2, 2]), label=r'Linblad Site 1', linewidth=2, linestyle=':')
ax.plot(times_100, pop_avg_traj_100[0, :])
ax.set_xlabel('Time', fontsize=12)
ax.set_ylabel('Population', fontsize=12)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()